<div class="alert alert-block alert-warning">
<b>Costs:</b> This QPU simulation costs ??? dollars.
</div>

<div class="alert alert-block alert-success">
<b>This simulation finds ground state energy for H2 molecule. </b>
<div>Obs. 1: The data is saved in external files so that it can be worked on without incurring new costs.</div>
<div>Obs. 2: For new simulation parameters, save in a file with a different name.</div>
</div>

# Instructions if you are running outside AWS cloud

1. Install aws-cli:

`sudo apt install aws-cli`

2. Setup aws credentials:

`aws configure --profile "default"`

3. Create virtual environment 

`python -m venv /path/to/new/virtual/environment`

4. Activate virtual environment

`source /path/to/new/virtual/environment/bin/activate`

In [ ]:
# Install this modules on your virtual environment
!pip install amazon-braket-sdk
!pip install boto3

# Another required packages
!pip install qiskit-nature
!pip install qiskit-aer
!pip install qiskit-ibm-runtime
!pip install qiskit_braket_provider
!pip install pyscf

In [47]:
!pip install pyscf

In [2]:
from boto3 import Session
from braket.aws import AwsSession

# Start Boto3 Session with your credentials profile
boto_session = Session(profile_name='default')

# Initialize Braket session with Boto3 Session credentials
aws_session = AwsSession(boto_session=boto_session)

# Defining the function to save results to json file and read results from file

In [3]:
import json

def save_to_file(data, file_name):
    with open(file_name+".json", "w") as file:
        json.dump(data, file, indent=4)

def read_from_file(file_name):
    with open(file_name+".json", 'r') as file:
        data = json.load(file)
    return data

# Bibliotecas

In [36]:
#Python Default 

import warnings

warnings.filterwarnings('ignore', category=DeprecationWarning)

import pickle
import time
import random

#Essentials

import numpy as np
from bs4 import BeautifulSoup
import requests
#import func_timeout
import pandas as pd
#from itables import show
from scipy.optimize import curve_fit

#from objproxies import *

import matplotlib.pyplot as plt

from scipy.optimize import curve_fit

#Qiskit

from qiskit.circuit import Gate
from qiskit.circuit.library import *
from qiskit import transpile
from qiskit.providers.models import *
from qiskit.providers.fake_provider import *
from qiskit.quantum_info import Statevector
from qiskit import QuantumCircuit
from qiskit_aer import StatevectorSimulator
import qiskit_aer

## Qiskit Nature
    
from qiskit_nature.units import *   #ANGSTROM

from qiskit_nature.second_q.drivers import *    #PySCFDriver

from qiskit_nature.second_q.mappers import * #JordanWignerMapper

from qiskit_nature.second_q.algorithms import GroundStateEigensolver

from qiskit_nature.second_q.problems import ElectronicStructureProblem
from qiskit_nature.second_q.problems import EigenstateResult

from qiskit_nature.second_q.circuit.library import *  #Ansatz, HF


## Qiskit Algorithms
    
from qiskit_algorithms.minimum_eigensolvers import VQE 
 
from qiskit_algorithms.optimizers import *    #SLSQP

from qiskit_algorithms.optimizers import L_BFGS_B

from qiskit.circuit.library import EfficientSU2   #EfficientSU2

## Qiskit Estimators

from qiskit.primitives import Estimator as Estimator_Nature

from qiskit_aer.primitives.estimator import Estimator as Estimator_Aer

#from qiskit_ibm_runtime import Estimator 

from qiskit_ibm_runtime import EstimatorV2 as Estimator

## Qiskit Noise Models

from qiskit_aer.noise import NoiseModel

from qiskit.providers.fake_provider import *

##Qiskit Runtime IBM

from qiskit_ibm_runtime import QiskitRuntimeService, Session, Options

## Braket

from braket.tracking import Tracker
from qiskit_braket_provider import *
from braket.aws import AwsDevice
from braket.devices import Devices
from braket.aws import AwsDevice, AwsQuantumTask


# %matplotlib widget
#%matplotlib inline

In [4]:
#IBMQ.save_account('YOUR API KEY', overwrite=True)

# Importando Backends - Computadores Quânticos

Backends são informações de Computadores Quânticos reais que nos ajudam na modelagem do noise

In [5]:
#Consulta os backends disponiveis para sua conta da AWSbraket
provider = BraketProvider()
available_cloud_backends = provider.backends() 
for i in available_cloud_backends: print(i)

BraketBackend[Aria 1]
BraketBackend[Aria 2]
BraketBackend[Aspen-M-3]
BraketBackend[Forte 1]
BraketBackend[Garnet]
BraketBackend[Harmony]


In [40]:
local_simulator = BraketLocalBackend()
statevector_simulator = StatevectorSimulator()
harmony = provider.get_backend("Harmony", aws_session=aws_session)
aria2 = provider.get_backend("Aria 2", aws_session=aws_session)
# harmony = AwsDevice(Devices.IonQ.Harmony, aws_session=aws_session)
# aria2 = AwsDevice(Devices.IonQ.Aria2, aws_session=aws_session)


In [8]:
print(local_simulator)

BraketBackend[default]


# Criando nosso problema de cálculo energético para molécula de hidrogênio - VQE

In [9]:
from qiskit_algorithms import NumPyMinimumEigensolver

#Definindo estimator - backend da braket e Mapper Jordan Wigner
#from qiskit.primitives import Estimator

estimator = Estimator_Nature()
mapper = JordanWignerMapper()
optimizer = L_BFGS_B()

numpy_solver = NumPyMinimumEigensolver()

driver = PySCFDriver(
    atom= f"H 0 0 0; H 0 0 0.7",
    basis="sto3g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)

h2_problem = driver.run()
#print(h2_problem)


In [11]:
ansatz = UCCSD(
    h2_problem.num_spatial_orbitals,
    h2_problem.num_particles,
    mapper,
    initial_state=HartreeFock(
        h2_problem.num_spatial_orbitals,
        h2_problem.num_particles,
        mapper,
    ),
)

vqe_solver = VQE(estimator, ansatz, optimizer)

vqe_solver.initial_point = [0] * ansatz.num_parameters

In [12]:
solver = GroundStateEigensolver(mapper, vqe_solver)

electronic_structure_result = solver.solve(h2_problem)

print(electronic_structure_result)

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -1.892156898237
  - computed part:      -1.892156898237
~ Nuclear repulsion energy (Hartree): 0.755967444171
> Total ground state energy (Hartree): -1.136189454066
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 2.000 S: 0.000 S^2: 0.000 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  1.32280829]
 
  0: 
  * Electronic dipole moment (a.u.): [0.0  0.0  1.322808313612]
    - computed part:      [0.0  0.0  1.322808313612]
  > Dipole moment (a.u.): [0.0  0.0  -0.000000023612]  Total: 0.000000023612
                 (debye): [0.0  0.0  -0.000000060015]  Total: 0.000000060015
 


In [16]:

minimum_eigenvalue = vqe_solver.compute_minimum_eigenvalue(solver.get_qubit_operators(h2_problem)[0])


Estou encontrando erro ao utilizar o EstimatorV2, então vou usar o Estimator_Nature por enquanto

# Criando Circuito baseado no Aria 1 - Amazon 

In [45]:
circuito = transpile(minimum_eigenvalue.optimal_circuit.decompose().assign_parameters(minimum_eigenvalue.optimal_parameters), aria2)

In [42]:
#task = aria2.run(circuito, shots=10)

/Users/chanahyo/Repository/Quantum Computing/lib/python3.12/site-packages/qiskit_braket_provider/providers/adapter.py:489: UserWarning: Device does not support global phase; global phase of 5.497787143782138 will not be included in Braket circuit
  warnings.warn(


In [46]:
task_id = task.job_id()

In [47]:
getData(task_id)

Status of (reconstructed) quantum task: QUEUED
Sorry, your quantum task is still being processed and has not been finalized yet.


0

/Users/chanahyo/Repository/Quantum Computing/lib/python3.12/site-packages/qiskit_braket_provider/providers/adapter.py:489: UserWarning: Device does not support global phase; global phase of 5.497787143782138 will not be included in Braket circuit
  warnings.warn(

In [34]:
def getData(aria1_task_id):
    # recover quantum task
    task_load = AwsQuantumTask(arn=aria1_task_id)

    # print status
    status = task_load.state()
    print('Status of (reconstructed) quantum task:', status)
    
    # wait for job to complete
    # terminal_states = ['COMPLETED', 'FAILED', 'CANCELLED']
    if status == 'COMPLETED':
        # get results
        aria1_results = task_load.result()

        # get all metadata of submitted quantum task
        metadata = task_load.metadata()
        
        # example for metadata
        shots = metadata['shots']
        machine = metadata['deviceArn']
        
        # print example metadata
        # print("{} shots taken on machine {}.\n".format(shots, machine))

        # get measurement counts
        aria1_counts = dict(sorted(aria1_results.measurement_counts.items()))
        print('Measurement counts:', aria1_counts)
        print('Expected value:', aria1_results.values[0])
        
        return aria1_results.values[0]

    elif status in ['FAILED', 'CANCELLED']:
        # print terminal message 
        print('Your quantum task is in terminal status, but has not completed.')
        return 0
    else:
        # print current status
        print('Sorry, your quantum task is still being processed and has not been finalized yet.')
        return 0

# Criando Circuito Baseado no Brisbane - IBM

In [11]:
# from qiskit import IBMQ (deprecated, need qiskit version 0.45)

In [ ]:
service = QiskitRuntimeService()
backend = service.backend("ibm_brisbane")
noise_model = NoiseModel.from_backend(backend)

circuito_ibm = transpile(minimum_eigenvalue.optimal_circuit.decompose().assign_parameters(minimum_eigenvalue.optimal_parameters), backend)

hamiltoniano = hproblem.second_q_ops()[0]

num_qubits_hamiltonian = JordanWignerMapper().map(hamiltoniano).num_qubits

new_circuit = QuantumCircuit(num_qubits_hamiltonian)

for gate in circuito_ibm.data:

    if all(circuito_ibm.qubits.index(qubit) < num_qubits_hamiltonian for qubit in gate[1]):
        new_circuit.append(gate[0], [circuito_ibm.qubits.index(qubit) for qubit in gate[1]])

circuito_ibm = new_circuit

# Definindo Funções - Modelos para criação de circuitos distintos na regressão de Clifford

Para escrever tais modelos vamos definir algumas funções para estes modelos

In [10]:
# O Clifford Gate produz uma rotação no eixo Z no estado quântico do qubit em um ângulo que seja múltiplo de pi/2

def clifford_gate():

   random_value = random.randint(0, 3)
   
   return RZGate(random_value*np.pi/2)

def cnot_gate(qr):
   
   control_qubit = random.randint(0, len(qr) - 1)
   
   target_qubit = random.choice([i for i in range(len(qr)) if i != control_qubit])
   
   return CXGate(), (control_qubit, target_qubit)

 
def crz_gate():
   
   random_value = random.randint(0, 3)

   crz_gate = CRZGate(random_value * np.pi / 2)

   return crz_gate

Além disso, podemos definir funções úteis para utilzação dos Simuladores

In [11]:
#Aqui estamos rodando nosso VQE classicamente

def run_statevector(circuit, driver, mapper = JordanWignerMapper()):
   
   hamiltoniano = driver.second_q_ops()[0]
   
   estado = Statevector.from_instruction(circuit)
   
   expectation_value = estado.expectation_value(mapper.map(hamiltoniano))
   
   return expectation_value.real

#Aqui estamos rodando o estimator que pode ou não ser um estimador quântico - Unitilizando Estimator V2

def run_estimator(circuit, driver, backend = None, shots = 2000, noise_model = None, coupling_map = None, mapper = JordanWignerMapper()):
   
   hamiltoniano = driver.second_q_ops()[0]
   
   num_qubits_hamiltonian = mapper.map(hamiltoniano).num_qubits
   num_qubits_circuit = circuit.num_qubits
   
   new_circuit = circuit.copy()
   
   if num_qubits_hamiltonian != num_qubits_circuit:

      new_circuit = QuantumCircuit(num_qubits_hamiltonian)

      for gate in circuit.data:

         if all(circuit.qubits.index(qubit) < num_qubits_hamiltonian for qubit in gate[1]):
            new_circuit.append(gate[0], [circuit.qubits.index(qubit) for qubit in gate[1]])
   
   circuit = new_circuit
   
   if noise_model is not None:
      
      estimator = Estimator(backend = backend, options = {"shots" : shots, "noise_model" : noise_model, "coupling_map" : coupling_map})
   
   else :
      
      estimator = Estimator(backend = backend, options = {"shots" : shots})
   
   job = estimator.run(circuit, mapper.map(hamiltoniano))
   
   return job.result().values[0]


#Aqui estamos rodando o estimator - Utilizando Estimator Aer

def run_estimator_aer(circuit, driver, shots = None, seed = None, mapper = JordanWignerMapper(), skip_transpilation = True):
   
   hamiltoniano = driver.second_q_ops()[0]
   
   num_qubits_hamiltonian = mapper.map(hamiltoniano).num_qubits
   num_qubits_circuit = circuit.num_qubits
   
   new_circuit = circuit.copy()
   
   if num_qubits_hamiltonian != num_qubits_circuit:

      new_circuit = QuantumCircuit(num_qubits_hamiltonian)

      for gate in circuit.data:

         if all(circuit.qubits.index(qubit) < num_qubits_hamiltonian for qubit in gate[1]):
            new_circuit.append(gate[0], [circuit.qubits.index(qubit) for qubit in gate[1]])
   
   circuit = new_circuit

   if shots != None:
   
      if seed is not None:
         
         estimator = Estimator_Aer(run_options = {"shots" : shots, "seed" : seed}, skip_transpilation = skip_transpilation)   
      
      else :
         
         estimator = Estimator_Aer(run_options = {"shots" : shots}, skip_transpilation = skip_transpilation)
   
   else :

      if seed is not None:
         
         estimator = Estimator_Aer(run_options = {"shots" : shots, "seed" : seed}, approximation = True, skip_transpilation = skip_transpilation)
      
      else :
         
         estimator = Estimator_Aer(run_options = {"shots" : shots}, approximation = True, skip_transpilation = skip_transpilation)
      
   
   job = estimator.run(circuit, mapper.map(hamiltoniano))
   
   return job.result().values[0]


#Aqui estamos rodando o estimator com simulação de ruído de uma maquina quântica - Utilizando Estimator Aer Noise


def run_estimator_aer_noise(circuit, driver, shots = None, seed = None, mapper = JordanWignerMapper(), skip_transpilation = True, noise_model = None, coupling_map = None):
   
   hamiltoniano = driver.second_q_ops()[0]
   
   num_qubits_hamiltonian = mapper.map(hamiltoniano).num_qubits
   num_qubits_circuit = circuit.num_qubits
   
   new_circuit = circuit.copy()
   
   if num_qubits_hamiltonian != num_qubits_circuit:

      new_circuit = QuantumCircuit(num_qubits_hamiltonian)

      for gate in circuit.data:

         if all(circuit.qubits.index(qubit) < num_qubits_hamiltonian for qubit in gate[1]):
            new_circuit.append(gate[0], [circuit.qubits.index(qubit) for qubit in gate[1]])
   
   circuit = new_circuit
   
   estimator = Estimator_Aer(backend_options={'noise_model': noise_model}, run_options = {"shots" : shots, "seed" : seed}, skip_transpilation = skip_transpilation)         
   
   job = estimator.run(circuit, mapper.map(hamiltoniano))
   
   return job.result().values[0]

ESTIMATORS = {
    'statevector': run_statevector,
    'estimator': run_estimator,
    'estimator_aer': run_estimator_aer,
    'estimator_aer_noise': run_estimator_aer_noise,
}

def check(original_value, target, driver, difference_percentage, estimator_name='statevector', **kwargs):
    estimator_function = ESTIMATORS[estimator_name]
    target_value = estimator_function(target, driver, **kwargs)
    return abs(original_value - target_value) < difference_percentage * abs(original_value), target_value

## Modelo 2

Tal modelo troca rotações Rz por rotações Rz de Clifford

In [12]:
def subs_cg_rz(quantum_circuit, n):
    
   quantum_circuit = quantum_circuit.copy()

   n = int(n * quantum_circuit.size())

   if n > quantum_circuit.size() + 1:
       
       raise ValueError("n muito grande.")

   permitted_gates = ["rz"]

   gates_to_change = random.sample(range(0, quantum_circuit.size()+1), n)

   for i, (gate, *qubits) in enumerate(quantum_circuit.data):
      
      if i in gates_to_change:
      
         if gate.name in permitted_gates:
            
            quantum_circuit.data[i] = (clifford_gate(), *qubits)

         else:
            
            new_gate = random.randint(i, len(quantum_circuit.data)-1)
            
            while quantum_circuit.data[new_gate][0].name not in permitted_gates:
                
                new_gate = random.randint(i, len(quantum_circuit.data)-1)

            quantum_circuit.data[new_gate] = (clifford_gate(), *quantum_circuit.data[new_gate][1:])

   return quantum_circuit

def modelo_2(original_circuit, number_of_circuits, driver, percentage_of_changed_gates=0.3, difference_percentage=0.5, do_check=True, estimator_name='statevector', **kwargs):
    
    original_circuit_exp_value = run_statevector(original_circuit, driver)
    
    while number_of_circuits > 0:
        new_circuit = subs_cg_rz(original_circuit, percentage_of_changed_gates)
        
        if do_check:
            check_result, target_value = check(original_circuit_exp_value, new_circuit, driver, difference_percentage, estimator_name=estimator_name, **kwargs)
            if check_result:
                number_of_circuits -= 1
                yield new_circuit, target_value
        else:
            number_of_circuits -= 1
            yield new_circuit
        


# Rodando Modelo 2

In [ ]:
modelo = modelo_2(circuito_ibm, 100, es_problem, percentage_of_changed_gates = 0.3, difference_percentage = 0.3, do_check=True, estimator_name = "estimator_aer", shots = None, seed = 1)

circuitos_similares_ibm = []

time_total, start_time = time.time(), time.time()

for index, i in enumerate(modelo):

    end_time = time.time() - start_time

    circuitos_similares_ibm.append(i)
    print(f"Iteração {index}: {end_time}s")

    start_time = time.time()
    
print(f"Tempo total: {time.time() - time_total}")